## telekom.de/hilfe

In [ ]:
# List of valid categoriy or folder names
folder_names = [
    "Geräte & Zubehör",
    "Hilfe bei Störungen",
    "Internet & Telefonie",
    "MagentaEINS",
    "Mobilfunk",
    "TV",
    "Vertrag & Rechnung",
    "Apps & Dienste"
]

# Define the mapping of category descriptions
category_map = {
    "vertrag_rechnung_ihre_daten_kundencenter_login-daten_rechnung_lieferstatus": "Vertrag & Rechnung",
    "hilfe_stoerungen_stoerungen_selbst_beheben_melden_status_verfolgen": "Hilfe bei Störungen",
    "mobilfunk_tarife_optionen_mobiles-internet_mailbox_esim_sim-karten": "Mobilfunk",
    "internet_telefonie:_ausbau,_sicherheit,_einstellungen,_bauherren,_glasfaser_und_wlan": "Internet & Telefonie",
    "tv_magentatv_streaming-dienste_magentatv_jugendschutz_pins": "TV",
    "magentains_kombi-pakete_mit_magentains_vorteil_und_treuebonus": "MagentaEINS",
    "apps_dienste_e-mail_magenta_apps_voicemail_app_mobilityconnect": "Apps & Dienste",
    "geraete_zubehoer_anleitungen_fuer_smartphones_tablets_telefone_router_receiver": "Geräte & Zubehör"
}

### Sitemap'ten qa pair'leri cekme ve txt dosyalarina yazdirma

In [7]:
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import re
import logging
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to create a session with retry logic
def create_session():
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
    return session

# 1. Fetch URLs from Sitemap
sitemap_url = 'https://www.telekom.de/ueber-das-unternehmen/robots/sitemap'
logger.info(f"Requesting sitemap URL: {sitemap_url}")
session = create_session()

try:
    response = session.get(sitemap_url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    logger.error(f"Error fetching sitemap: {e}")
    exit(1)

# Parse XML sitemap
soup = BeautifulSoup(response.content, 'xml')
logger.info("Processing sitemap XML...")

# Extract URLs
urls = [url.text for url in soup.find_all('loc') if url is not None]
logger.info(f"{len(urls)} URLs found.")

# Function to extract question-answer pairs from the page
def extract_question_answer(soup):
    question_answer_pairs = []

    def get_text_from_element(element):
        text = ''
        for p in element.find_all('p'):
            text += p.get_text(strip=True) + '\n'
        for ul in element.find_all('ul'):
            for li in ul.find_all('li'):
                text += f"• {li.get_text(strip=True)}\n"
        return text.strip()

    def extract_accordion_items(accordion_list):
        for item in accordion_list.find_all('li', class_='accordion-item'):
            question = item.find('p', class_='accordion-item__title')
            answer = item.find('div', class_='accordion-item__content')
            if question and answer:
                question_text = question.get_text(strip=True)
                answer_text = get_text_from_element(answer)
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    excluded_classes = [
        "chf-navigation-bar",
        "direct-access-container",
        "direct-access-content",
        "collection-wrapper collection collection-standard",
        "collection-wrapper collection collection-standard l-outer l-outer--solutionPage"
    ]
    
    def remove_excluded_elements(soup):
        for class_name in excluded_classes:
            for element in soup.find_all(class_=class_name):
                if element:
                    element.decompose()

    def remove_nested_excluded_elements(soup):
        for element in soup.find_all(True):
            if isinstance(element, BeautifulSoup):
                classes = element.get('class', [])
                if classes and any(cls in ' '.join(classes) for cls in excluded_classes):
                    element.decompose()
    
    remove_excluded_elements(soup)
    remove_nested_excluded_elements(soup)

    questions = soup.find_all(['h1', 'h2', 'h3'])
    for question in questions:
        question_text = question.get_text(strip=True)
        if question_text.endswith('?'):
            answer_text = ''
            next_div = question.find_next('div', class_='outerRichtextDiv')
            if next_div:
                answer_text = get_text_from_element(next_div)
            if not answer_text:
                next_div = question.find_next('div')
                if next_div and not any(cls in ' '.join(next_div.get('class', [])) for cls in excluded_classes):
                    answer_text = get_text_from_element(next_div)
            if answer_text:
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    accordion_list = soup.find('ul', class_='accordion-list')
    if accordion_list:
        extract_accordion_items(accordion_list)

    return question_answer_pairs

# 3. Process each URL
output_dir = Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

for idx, url in enumerate(urls, 1):
    logger.info(f"Processing URL {idx}/{len(urls)}: {url}")
    
    try:
        response = session.get(url, allow_redirects=False)
        
        if response.status_code in {301, 302}:
            logger.info("   Redirect detected, checking URL.")
            final_url = response.headers.get('Location')
            if final_url:
                response = session.get(final_url)
        elif response.status_code != 200:
            logger.warning("   Invalid URL or access problem.")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        qa_pairs = extract_question_answer(soup)
        logger.info(f"   {len(qa_pairs)} question-answer pairs found.")
        
        if qa_pairs:
            # Create a file name using the URL and replace forbidden characters
            file_name = url.replace(":", "_").replace("/", "_") + ".txt"
            output_file = output_dir / file_name
            
            try:
                with open(output_file, "w", encoding="utf-8") as file:
                    file.write(f"Source URL: {url}\n\n")
                    for qa in qa_pairs:
                        file.write(f"Question: {qa['question']}\nAnswer: {qa['answer']}\n\n")
                
                logger.info(f"   Results saved to '{output_file}'.")
            except IOError as e:
                logger.error(f"   Error writing to file: {e}")
        else:
            logger.info("   No question-answer pairs found, file will not be created.")
    
    except requests.exceptions.RequestException as e:
        logger.error(f"   Error occurred: {e}")
    
    time.sleep(1)  # Sleep to avoid overwhelming the server

logger.info("Processing completed! Results for pages with question-answer pairs saved in 'data' directory.")

2024-10-20 17:27:00,561 - INFO - Requesting sitemap URL: https://www.telekom.de/ueber-das-unternehmen/robots/sitemap
2024-10-20 17:27:00,888 - INFO - Processing sitemap XML...
2024-10-20 17:27:00,893 - INFO - 4866 URLs found.
2024-10-20 17:27:00,894 - INFO - Processing URL 1/4866: https://www.telekom.de/apps
2024-10-20 17:27:00,986 - INFO -    Redirect detected, checking URL.
2024-10-20 17:27:01,490 - INFO -    0 question-answer pairs found.
2024-10-20 17:27:01,490 - INFO -    No question-answer pairs found, file will not be created.
2024-10-20 17:27:02,495 - INFO - Processing URL 2/4866: https://www.telekom.de/apps/meinmagenta-app
2024-10-20 17:27:02,818 - INFO -    0 question-answer pairs found.
2024-10-20 17:27:02,818 - INFO -    No question-answer pairs found, file will not be created.
2024-10-20 17:27:03,824 - INFO - Processing URL 3/4866: https://www.telekom.de/apps/voicemail
2024-10-20 17:27:04,210 - INFO -    0 question-answer pairs found.
2024-10-20 17:27:04,210 - INFO -    No

## telekomhilft.telekom.de

### Community url toplama

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# Safari WebDriver başlat
driver = webdriver.Safari()

# Hedef URL
url = "https://telekomhilft.telekom.de/?ms-sort=most_viewed&ms-messagestatusfilter=solved"
driver.get(url)

# Sayfanın yüklenmesi için bekleyelim
time.sleep(5)

# Mesaj URL'lerini tutmak için boş bir liste
message_urls = []

# Kaydırma sayısını ayarlayın
max_messages = 1000  # Maksimum mesaj sayısı

# "Weitere Beiträge laden" butonu görünene kadar döngü
while len(message_urls) < max_messages:
    # Sayfanın en altına kaydır
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Kaydırma sonrası biraz bekleyelim

    # Sayfanın HTML kaynağını alalım
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Mesajların URL'lerini bulalım
    for div_tag in soup.find_all('div', class_='thc-message-item contenttype-forum'):
        a_tag = div_tag.find('a', class_='telekom-forum-message-link')
        if a_tag:
            link = a_tag.get('href')
            if link.startswith('/'):
                full_url = "https://telekomhilft.telekom.de" + link
            else:
                full_url = link
            if full_url not in message_urls:  # Tekrar eden URL'leri önleyelim
                message_urls.append(full_url)
                
            # Eğer 500 URL'ye ulaşıldıysa döngüyü sonlandır
            if len(message_urls) >= max_messages:
                break  # İç içe döngüden çık

    # Eğer 500 URL'ye ulaşıldıysa ana döngüden çık
    if len(message_urls) >= max_messages:
        break

    # "Weitere Beiträge laden" butonunu kontrol et
    try:
        load_more_button = driver.find_element(By.XPATH, "//a[contains(@class, 'telekom-forum-pager-next')]")
        
        # Buton görünür mü kontrol et
        if load_more_button.is_displayed():
            # Eğer buton görünüyorsa tıklayın
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(5)  # Yeni mesajlar yüklenirken bekleyelim
        else:
            break  # Eğer buton görünmüyorsa döngüden çık

    except Exception as e:
        print("Buton bulunamadı veya başka bir hata meydana geldi. Hata mesajı:", str(e))
        break

# Selenium tarayıcısını kapat
driver.quit()

# URL'leri bir dosyaya yazdıralım
with open('message_urls.txt', 'w') as file:
    for message_url in message_urls:
        file.write(message_url + '\n')

print(f"{len(message_urls)} mesaj URL'si bulundu ve 'message_urls.txt' dosyasına yazıldı.")

933 mesaj URL'si bulundu ve 'message_urls.txt' dosyasına yazıldı.


### Data yi txt olarak yazdirma

In [6]:
import requests
from bs4 import BeautifulSoup
import os
import re

# URL file
url_file = 'message_urls.txt'
# Output directory
output_dir = 'data'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to create a valid filename from a URL
def create_filename(url):
    # Remove protocol and query parameters, then replace invalid characters
    filename = re.sub(r'https?://', '', url).split('?')[0]  # Remove 'http://' or 'https://'
    filename = filename.replace('/', '_')  # Replace '/' with '_'
    return filename

# Read URLs from the file
with open(url_file, 'r') as file:
    urls = [line.strip() for line in file.readlines()]

# Define texts to exclude
exclude_texts = ["Gelöst!", "Gehe zu Lösung.", "Lösung in ursprünglichem Beitrag anzeigen"]

# Function to filter out excluded texts
def filter_excluded_texts(text):
    for exclude in exclude_texts:
        text = text.replace(exclude, "")
    return text.strip()

# Process each URL
for url in urls:
    try:
        # Get content from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Process HTML content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the topic title
        topic_div = soup.find('div', class_='topic-message')
        topic_title = topic_div.find('h1', class_='topic-subject').get_text(strip=True) if topic_div else 'Topic not found'

        # Find the topic content
        topic_content = ''
        content_div = soup.find('div', class_='lia-message-body-content')
        if content_div:
            topic_content = content_div.get_text(strip=True)

        # Find accepted solutions
        solutions = []
        accepted_solutions = soup.find_all('div', class_='lia-message-body-content')
        for solution in accepted_solutions:
            if solution.find('div', class_='lia-message-body-accepted-solution-checkmark'):
                solution_text = solution.get_text(strip=True)
                filtered_solution = filter_excluded_texts(solution_text)
                if filtered_solution:  # Only add non-empty solutions
                    solutions.append(filtered_solution)

        # Create filename based on URL
        filename = create_filename(url)
        output_file_path = os.path.join(output_dir, f"{filename}.txt")
        
        # Write results to a file
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(f"Source URL: {url}\n")  # Write the source URL at the top
            output_file.write(f"Subject: {topic_title}\n")
            output_file.write(f"Content: {filter_excluded_texts(topic_content)}\n\n")  # Filter content
            output_file.write("Accepted Solutions:\n")
            for sol in solutions:
                output_file.write(f"- {sol}\n")
        
        print(f"{output_file_path} has been written.")

    except Exception as e:
        print(f"An error occurred ({url}): {e}")

data/telekomhilft.telekom.de_t5_Mobilfunk_Wann-kommt-5G-SA-gibt-es-da-neue-Infos_td-p_6945413_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_Sonstiges_Intervall-Stoerung_td-p_6956428_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_TV_Seit-Gestern-Lautstaerke-nur-von-RTL-Sendern-extrem-leise_td-p_6949692_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_TV_Sky-Highlights-nur-in-englischer-Sprache_td-p_6959933_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_Mobilfunk_iPhone-16-Pro-Max-Versand_td-p_6947273_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_Mobilfunk_Lieferstatus-iPhone-16-Pro_td-p_6962729_jump-to_first-unread-message.txt has been written.
data/telekomhilft.telekom.de_t5_Festnetz-Internet_Kulanzhilfe-von-Telekom-Werbern_td-p_6941739_jump-to_first-unread-message.txt has been written.
data/telekomhilft.tele

### Kategorize etme

In [8]:
import os
import shutil
from dotenv import load_dotenv
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

# Load API key from .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the GPT-4 model
model = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

# Categories
folder_names = [
    "Geräte & Zubehör",
    "Hilfe bei Störungen",
    "Internet & Telefonie",
    "MagentaEINS",
    "Mobilfunk",
    "TV",
    "Vertrag & Rechnung",
    "Apps & Dienste"
]

# Directories
source_dir = "/Users/taha/Desktop/rag/rag_data/web/data"  # Source directory where txt files are stored
target_base_dir = "/Users/taha/Desktop/rag/data"  # Target directory where files will be copied

# Create directories in the target base directory
def create_directories():
    for category in folder_names:
        category_dir = os.path.join(target_base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

# Function to ask GPT-4 for categorization based on file content
def classify_with_gpt(filename, content, model, folder_names):
    # Create the prompt with the folder names
    prompt = f"""
    I have a file named '{filename}' and its content is as follows:

    {content}

    Based on the filename and the content, which three of the following categories best fit this file?
    Categories:
    - {', '.join(folder_names)}

    Please reply with exactly three categories from the list, separated by commas.
    """

    try:
        # Send the prompt to GPT-4 and retrieve the response
        response = model([HumanMessage(content=prompt)])
        categories = response.content.strip().split(", ")
        return categories
    except Exception as e:
        print(f"Error during GPT classification: {e}")
        return None

# Read file content in chunks to avoid token limit
def read_file_content(file_path, chunk_size=8192):
    content = ""
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            content += chunk
    return content

# Main function to classify and copy files
def categorize_files():
    for filename in os.listdir(source_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(source_dir, filename)
            
            # Read file content
            file_content = read_file_content(file_path)

            # Use GPT-4 to determine the 3 best fitting categories
            categories = classify_with_gpt(filename, file_content, model, folder_names)

            if categories:
                for category in categories:
                    if category in folder_names:
                        category_dir = os.path.join(target_base_dir, category)
                        destination_path = os.path.join(category_dir, filename)

                        # Copy file to each of the 3 categories
                        if not os.path.exists(destination_path):
                            shutil.copy(file_path, destination_path)
                            print(f"{filename} has been copied to {category} category.")
                        else:
                            print(f"{filename} already exists in {category} category.")
            else:
                print(f"Could not categorize {filename}.")

# Main execution
def main():
    create_directories()  # Create the necessary category folders
    categorize_files()    # Categorize and copy the files

if __name__ == "__main__":
    main()

2024-10-20 19:57:32,557 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


https___www.telekom.de_hilfe_geraete-zubehoer_telefone-und-anlagen_yealink-telefone-und-zubehoer_yealink-tischtelefone-g-serie_unterschied-g-serie-zu-s-serie.txt has been copied to Geräte & Zubehör category.
https___www.telekom.de_hilfe_geraete-zubehoer_telefone-und-anlagen_yealink-telefone-und-zubehoer_yealink-tischtelefone-g-serie_unterschied-g-serie-zu-s-serie.txt has been copied to Hilfe bei Störungen category.
https___www.telekom.de_hilfe_geraete-zubehoer_telefone-und-anlagen_yealink-telefone-und-zubehoer_yealink-tischtelefone-g-serie_unterschied-g-serie-zu-s-serie.txt has been copied to Internet & Telefonie category.


2024-10-20 19:57:33,476 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


https___www.telekom.de_hilfe_geraete-zubehoer_handy-smartphone-tablet_iphone-7_software-version-ios-iphone-7.txt has been copied to Geräte & Zubehör category.
https___www.telekom.de_hilfe_geraete-zubehoer_handy-smartphone-tablet_iphone-7_software-version-ios-iphone-7.txt has been copied to Hilfe bei Störungen category.
https___www.telekom.de_hilfe_geraete-zubehoer_handy-smartphone-tablet_iphone-7_software-version-ios-iphone-7.txt has been copied to Mobilfunk category.


2024-10-20 19:57:33,990 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

telekomhilft.telekom.de_t5_Festnetz-Internet_festnetzrechnungen-2022-einsehen_td-p_6945665_jump-to_first-unread-message.txt has been copied to Vertrag & Rechnung category.
telekomhilft.telekom.de_t5_Festnetz-Internet_festnetzrechnungen-2022-einsehen_td-p_6945665_jump-to_first-unread-message.txt has been copied to Internet & Telefonie category.
telekomhilft.telekom.de_t5_Festnetz-Internet_festnetzrechnungen-2022-einsehen_td-p_6945665_jump-to_first-unread-message.txt has been copied to Hilfe bei Störungen category.


2024-10-20 20:23:34,172 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


telekomhilft.telekom.de_t5_Festnetz-Internet_Schlechte-Internetleistung-trotz-Magenta-Zuhause-XL-Falsche_td-p_6960404_jump-to_first-unread-message.txt has been copied to Geräte & Zubehör category.
telekomhilft.telekom.de_t5_Festnetz-Internet_Schlechte-Internetleistung-trotz-Magenta-Zuhause-XL-Falsche_td-p_6960404_jump-to_first-unread-message.txt has been copied to Hilfe bei Störungen category.
telekomhilft.telekom.de_t5_Festnetz-Internet_Schlechte-Internetleistung-trotz-Magenta-Zuhause-XL-Falsche_td-p_6960404_jump-to_first-unread-message.txt has been copied to Internet & Telefonie category.


2024-10-20 20:23:34,895 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


https___www.telekom.de_unterwegs_apple_apple-iphone-15-plus.txt has been copied to Geräte & Zubehör category.
https___www.telekom.de_unterwegs_apple_apple-iphone-15-plus.txt has been copied to Mobilfunk category.
https___www.telekom.de_unterwegs_apple_apple-iphone-15-plus.txt has been copied to Apps & Dienste category.
